# MNIST Dataset with Tensorflow API 

In [1]:
import tensorflow as tf 
import numpy as np 

import struct
 
def load_mnist(path, kind='train'):
    
    if kind == 'train' : 
        
        labels_path = 'train-labels.idx1-ubyte'
        images_path = 'train-images.idx3-ubyte'
    else : 
        labels_path = 't10k-labels.idx1-ubyte'
        images_path = 't10k-images.idx3-ubyte'
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', 
                                 lbpath.read(8))
        labels = np.fromfile(lbpath, 
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", 
                                               imgpath.read(16))
        images = np.fromfile(imgpath, 
                             dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
 
    return images, labels

In [2]:
X_train, y_train = load_mnist('./', kind='train')
print('Rows: %d,  Columns: %d' %(X_train.shape[0], 
                                 X_train.shape[1]))
X_test, y_test = load_mnist('./', kind='t10k')
print('Rows: %d,  Columns: %d' %(X_test.shape[0], 
                                 X_test.shape[1]))

Rows: 60000,  Columns: 784
Rows: 10000,  Columns: 784


In [40]:
mean_vals = np.mean(X_train,axis=0)

std_dev = np.std(X_train)

X_train_centered = (X_train-mean_vals)/std_dev

X_test_centered = (X_test-mean_vals)/std_dev

In [42]:
n_features = X_train_centered.shape[1]

n_classes = 10 

random_seed = 123 

np.random.seed(random_seed)

In [43]:
g = tf.Graph()

In [44]:
with g.as_default() : 
    
    tf.set_random_seed(random_seed)
    
    tf_x = tf.placeholder(shape=(None,n_features),dtype=tf.float32,name="tf_x")
    tf_y = tf.placeholder(shape=None,dtype=tf.int32,name="tf_y")
    
    y_onehot = tf.one_hot(indices=tf_y,depth=n_classes)
    
    h1 = tf.layers.dense(inputs=tf_x,activation=tf.tanh,units=50,name="layers")
    
    h2 = tf.layers.dense(inputs=h1,activation=tf.tanh,name="layer2",units=50)
    
    logits = tf.layers.dense(inputs=h2,units=10,activation=None,name="output_layer")
    
    predictions = {'classes':tf.argmax(logits,axis=1,name="predicted_classes"),
                    'probs':tf.nn.softmax(logits,name="softmax_tensor")
                  }

In [46]:
with g.as_default() : 
    
    cost = tf.losses.softmax_cross_entropy(onehot_labels=y_onehot,logits=logits)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    
    train_op = optimizer.minimize(loss=cost)
    
    init_op = tf.global_variables_initializer()
    

In [47]:
def create_batch_generator(X, y, batch_size=128, shuffle=False):
    X_copy = np.array(X)
    y_copy = np.array(y)
    
    if shuffle:
        data = np.column_stack((X_copy, y_copy))
        np.random.shuffle(data)
        X_copy = data[:, :-1]
        y_copy = data[:, -1].astype(int)
    
    for i in range(0, X.shape[0], batch_size):
        yield (X_copy[i:i+batch_size, :], y_copy[i:i+batch_size])

In [48]:
sess = tf.Session(graph=g)

In [49]:
sess.run(init_op)

In [50]:
training_costs = []
for epoch in range(50):
    training_loss = []
    batch_generator = create_batch_generator(
            X_train_centered, y_train, 
            batch_size=64)
    for batch_X, batch_y in batch_generator:
        ## prepare a dict to feed data to our network:
        feed = {tf_x:batch_X, tf_y:batch_y}
        _, batch_cost = sess.run([train_op, cost],
                                 feed_dict=feed)
        training_costs.append(batch_cost)
    print(' -- Epoch %2d  '
          'Avg. Training Loss: %.4f' % (
              epoch+1, np.mean(training_costs)
    ))

 -- Epoch  1  Avg. Training Loss: 1.5573
 -- Epoch  2  Avg. Training Loss: 1.2532
 -- Epoch  3  Avg. Training Loss: 1.0854
 -- Epoch  4  Avg. Training Loss: 0.9738
 -- Epoch  5  Avg. Training Loss: 0.8924
 -- Epoch  6  Avg. Training Loss: 0.8296
 -- Epoch  7  Avg. Training Loss: 0.7794
 -- Epoch  8  Avg. Training Loss: 0.7381
 -- Epoch  9  Avg. Training Loss: 0.7032
 -- Epoch 10  Avg. Training Loss: 0.6734
 -- Epoch 11  Avg. Training Loss: 0.6475
 -- Epoch 12  Avg. Training Loss: 0.6247
 -- Epoch 13  Avg. Training Loss: 0.6045
 -- Epoch 14  Avg. Training Loss: 0.5864
 -- Epoch 15  Avg. Training Loss: 0.5700
 -- Epoch 16  Avg. Training Loss: 0.5551
 -- Epoch 17  Avg. Training Loss: 0.5415
 -- Epoch 18  Avg. Training Loss: 0.5290
 -- Epoch 19  Avg. Training Loss: 0.5175
 -- Epoch 20  Avg. Training Loss: 0.5068
 -- Epoch 21  Avg. Training Loss: 0.4968
 -- Epoch 22  Avg. Training Loss: 0.4875
 -- Epoch 23  Avg. Training Loss: 0.4788
 -- Epoch 24  Avg. Training Loss: 0.4706
 -- Epoch 25  Av

In [51]:
feed = {tf_x : X_test_centered}
y_pred = sess.run(predictions['classes'], 
                  feed_dict=feed)
 
print('Test Accuracy: %.2f%%' % (
      100*np.sum(y_pred == y_test)/y_test.shape[0]))

Test Accuracy: 93.89%


# MNIST Dataset with Keras API 

In [63]:
import keras

In [64]:
mean_vals = np.mean(X_train,axis=0)

std_val = np.std(X_train)

NameError: name 'X_train' is not defined

In [ ]:
X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val
 
del X_train, X_test
 
print(X_train_centered.shape, y_train.shape)

print(X_test_centered.shape, y_test.shape)

In [ ]:
print('First 3 labels: ', y_train[:3])
print('\nFirst 3 labels (one-hot):\n', y_train_onehot[:3])

In [67]:
model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        units=50,    
        input_dim=X_train_centered.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense(
        units=50,    
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model.add(
    keras.layers.Dense(
        units=y_train_onehot.shape[1],    
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))


sgd_optimizer = keras.optimizers.SGD(
        lr=0.001, decay=1e-7, momentum=.9)

model.compile(optimizer=sgd_optimizer,
              loss='categorical_crossentropy')

In [68]:
history = model.fit(X_train_centered, y_train_onehot,
                    batch_size=64, epochs=50,
                    verbose=1,
                    validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 3s 47us/step - loss: 0.7247 - val_loss: 0.3621
Epoch 2/50
54000/54000 [==============================] - 3s 47us/step - loss: 0.3657 - val_loss: 0.2718
Epoch 3/50
54000/54000 [==============================] - 4s 66us/step - loss: 0.2970 - val_loss: 0.2331
Epoch 4/50
54000/54000 [==============================] - 3s 60us/step - loss: 0.2604 - val_loss: 0.2105
Epoch 5/50
54000/54000 [==============================] - 3s 58us/step - loss: 0.2353 - val_loss: 0.1930
Epoch 6/50
54000/54000 [==============================] - 3s 50us/step - loss: 0.2162 - val_loss: 0.1807
Epoch 7/50
54000/54000 [==============================] - 3s 56us/step - loss: 0.2006 - val_loss: 0.1718
Epoch 8/50
54000/54000 [==============================] - 3s 50us/step - loss: 0.1877 - val_loss: 0.1634
Epoch 9/50
54000/54000 [==============================] - 3s 51us/step - loss: 0.1763 - val_loss: 0.1565
Epoch 

In [69]:
y_train_preds = model.predict_classes(X_train_centered,verbose=0)

In [70]:
correct_train_predictions = np.sum(y_train==y_train_preds,axis=0)

In [71]:
training_accuracy = correct_train_predictions/y_train.shape[0]

In [72]:
training_accuracy*100

98.97833333333334

In [73]:
y_test_predictions= model.predict_classes(X_test_centered,verbose=0)

In [74]:
correct_test_preds = np.sum(y_test==y_test_predictions,axis=0)

In [75]:
testing_accuracy = correct_test_preds/y_test.shape[0]

In [76]:
testing_accuracy*100

96.1